# Titanic: Predicting who survived using Random Forests

In [135]:
using CSV
using DataFrames
using DataFramesMeta
using DecisionTree
using StatsBase
using TypedTables
using Plots
plotly()
IJulia.clear_output() ## to suppress all warnings

0

## Descriptive Analysis
Let's read in some data and have a look at it.

In [136]:
train = CSV.read("train.csv", rows_for_type_detect=200);
test = CSV.read("test.csv", rows_for_type_detect=200);

print(names(train), "\n")
print(names(test))

Symbol[:PassengerId, :Survived, :Pclass, :Name, :Sex, :Age, :SibSp, :Parch, :Ticket, :Fare, :Cabin, :Embarked]
Symbol[:PassengerId, :Pclass, :Name, :Sex, :Age, :SibSp, :Parch, :Ticket, :Fare, :Cabin, :Embarked]

### Distribution of survival:

In [137]:
print("Number of passengers survived: ", countmap(train[:Survived])[1], "\n",
      "Number of passengers died: ", countmap(train[:Survived])[0])

Number of passengers survived: 342
Number of passengers died: 549

### Distribution of Passenger classes booked:

In [138]:
print("Class 1: ", countmap(train[:Pclass])[1], "\n",
      "Class 2: ", countmap(train[:Pclass])[2], "\n",
      "Class 3: ", countmap(train[:Pclass])[3])

Class 1: 216
Class 2: 184
Class 3: 491

### Distribution of gender:

In [139]:
print("Number of male passengers: ", countmap(train[:Sex])["male"], "\n",
      "Number of female passengers: ", countmap(train[:Sex])["female"])

Number of male passengers: 577
Number of female passengers: 314

### Distribution of age:

In [140]:
describe(train[:Age])

Summary Stats:
Mean:           29.699118
Minimum:        0.420000
1st Quartile:   20.125000
Median:         28.000000
3rd Quartile:   38.000000
Maximum:        80.000000
Length:         891
Type:           Union{Float64, Nulls.Null}
Number Missing: 177
% Missing:      19.865320


### Distribution of relatives on board

In [141]:
print("Number of siblings/spouses on board: ")
countmap(train[:SibSp])

Number of siblings/spouses on board: 

Dict{Int64,Int64} with 7 entries:
  0 => 608
  4 => 18
  2 => 28
  3 => 16
  5 => 5
  8 => 7
  1 => 209

In [142]:
print("Number of parents/children aboard")
countmap(train[:Parch])

Number of parents/children aboard

Dict{Int64,Int64} with 7 entries:
  0 => 678
  4 => 4
  2 => 80
  3 => 5
  5 => 5
  6 => 1
  1 => 118

In [143]:
describe(train[:Fare])

Summary Stats:
Mean:           32.204208
Minimum:        0.000000
1st Quartile:   7.910400
Median:         14.454200
3rd Quartile:   31.000000
Maximum:        512.329200
Length:         891
Type:           Float64


In [144]:
describe(train[:Name])

Summary Stats:
Length:         891
Type:           WeakRefString{UInt8}
Number Unique:  891


# Data transformations
## Filter null values
Null.null values are taken out here because the build_forest procedure called later does not know how to handle them.

In [189]:
train = @where(train, !(:Age.===null));
train = @where(train, !(:Sex.===null));
train = @where(train, !(:Pclass.===null));
train = @where(train, !(:SibSp.===null));
train = @where(train, !(:Parch.===null));
train = @where(train, !(:Fare.===null));
train = @where(train, !(:Embarked.===null));
train = @where(train, !(:Name.===null));

train[:Cabin] = collect(Nulls.replace(train[:Cabin], ""));
test[:Cabin] = collect(Nulls.replace(test[:Cabin], ""));

## Convert categorical features to ordered categoricals

In [146]:
ordered!(train[:Sex], true);
ordered!(test[:Sex], true);

train = @orderby(train, :Age);
train[:Age] = string.(train[:Age]);
train[:Age] = CategoricalArray(train[:Age]);
ordered!(train[:Age], true);

test = @orderby(test, :Age);
test[:Age] = string.(test[:Age]);
test[:Age] = CategoricalArray(test[:Age]);
ordered!(test[:Age], true);

train = @orderby(train, :Pclass);
train[:Pclass] = string.(train[:Pclass]);
train[:Pclass] = CategoricalArray(train[:Pclass]);
ordered!(train[:Pclass], true);

test = @orderby(test, :Pclass);
test[:Pclass] = string.(test[:Pclass]);
test[:Pclass] = CategoricalArray(test[:Pclass]);
ordered!(test[:Pclass], true);

train = @orderby(train, :SibSp);
train[:SibSp] = string.(train[:SibSp]);
train[:SibSp] = CategoricalArray(train[:SibSp]);
ordered!(train[:SibSp], true);

test = @orderby(test, :SibSp);
test[:SibSp] = string.(test[:SibSp]);
test[:SibSp] = CategoricalArray(test[:SibSp]);
ordered!(test[:SibSp], true);

train = @orderby(train, :Parch);
train[:Parch] = string.(train[:Parch]);
train[:Parch] = CategoricalArray(train[:Parch]);
ordered!(train[:Parch], true);

test = @orderby(test, :Parch);
test[:Parch] = string.(test[:Parch]);
test[:Parch] = CategoricalArray(test[:Parch]);
ordered!(test[:Parch], true);

train = @orderby(train, :Fare);
train[:Fare] = string.(train[:Fare]);
train[:Fare] = CategoricalArray(train[:Fare]);
ordered!(train[:Fare], true);

test = @orderby(test, :Fare);
test[:Fare] = string.(test[:Fare]);
test[:Fare] = CategoricalArray(test[:Fare]);
ordered!(test[:Fare], true);

train = @orderby(train, :Cabin);
train[:Cabin] = string.(train[:Cabin]);
train[:Cabin] = CategoricalArray(train[:Cabin]);
ordered!(train[:Cabin], true);

test = @orderby(test, :Cabin);
test[:Cabin] = string.(test[:Cabin]);
test[:Cabin] = CategoricalArray(test[:Cabin]);
ordered!(test[:Cabin], true);

train = @orderby(train, :Embarked);
train[:Embarked] = string.(train[:Embarked]);
train[:Embarked] = CategoricalArray(train[:Embarked]);
ordered!(train[:Embarked], true);

test = @orderby(test, :Embarked);
test[:Embarked] = string.(test[:Embarked]);
test[:Embarked] = CategoricalArray(test[:Embarked]);
ordered!(test[:Embarked], true);


## Feature generation

In the following we extract two features from the data that may have predictive value. Here it's merely an exercise and the predictive value remains untested. Usually I would want to know, though.

Firstly, we extract the title from the name.

In [147]:
train = @byrow! train begin
    @newcol Title::Array{String}
    :Title=""
    if(ismatch(r"Mr\.", String(:Name)))
        :Title="Mr"
    end
    if(ismatch(r"Mrs\.", String(:Name)))
        :Title="Mrs"
    end
    if(ismatch(r"Miss\.", String(:Name)))
        :Title="Miss"
    end
    if(ismatch(r"master|Master", String(:Name)))
        :Title="Master"
    end
end;


test = @byrow! test begin
    @newcol Title::Array{String}
    :Title=""
    if(ismatch(r"Mr\.", String(:Name)))
        :Title="Mr"
    end
    if(ismatch(r"Mrs\.", String(:Name)))
        :Title="Mrs"
    end
    if(ismatch(r"Miss\.", String(:Name)))
        :Title="Miss"
    end
    if(ismatch(r"master|Master", String(:Name)))
        :Title="Master"
    end
end;

countmap(train[:Title])

Dict{String,Int64} with 5 entries:
  "Miss"   => 145
  "Master" => 36
  ""       => 26
  "Mr"     => 398
  "Mrs"    => 107

As another feature we extract the fact that some of the passengers had booked a private Cabin and may have been given precedence when deciding on whom to save first.

In [148]:
train = @byrow! train begin
    @newcol Cabin_Booked::Array{String}
    :Cabin_Booked="No"
    if(ismatch(r"[a-zA-Z0-9]+", String(:Cabin)))
        :Cabin_Booked="Yes"
    end
end;

test = @byrow! test begin
    @newcol Cabin_Booked::Array{String}
    :Cabin_Booked="No"
    if(ismatch(r"[a-zA-Z0-9]+", String(:Cabin)))
        :Cabin_Booked="Yes"
    end
end;

countmap(train[:Cabin_Booked])

Dict{String,Int64} with 2 entries:
  "Yes" => 183
  "No"  => 529

## Splitting training and cross-validation sets

Here wer create a training and a validation set with
 - n = 570 passengers (~80%) in training set and
 - n = 142 (~20%) in cross validation set

In [149]:
n = size(train)[1];
idx = sample(1:n, 570, replace=false, ordered=true);
t = fill(true, n);
t[idx] = false; ## true = cv, false = train

cv_set = train[t, :];
train_set = train[!t, :];

Assert that procedure succeeded by checking that no passenger id from cv_set exists in train_set.

In [150]:
any([cv_set[i, :PassengerId] in train_set[:PassengerId] for i in 1:size(cv_set)[1]])

false

## Create data sets for model

In [151]:
feature_list=[:Pclass, :Age, :Sex, :Parch, :SibSp, :Fare, :Embarked, :Title, :Cabin_Booked];

labels = convert(Vector, train[:,:Survived]);
features = convert(Matrix, train[:,feature_list]);

labels_train = convert(Vector, train_set[:,:Survived]);
features_train = convert(Matrix, train_set[:,feature_list]);

labels_cv = convert(Vector, cv_set[:,:Survived]);
features_cv = convert(Matrix, cv_set[:,feature_list]);

## Tuning the hyperparameters

The following steps test different values for the following hyperparameters used in the random forest training

- number of features per sample
- number of trees built
- sample size as a percentage of the full sample
- depth of the trees fitted

in order to build the model.

The values for the hyperparameters are saved in the following variables:

 - nf = number of features
 - nt = number of trees
 - p = portion of the sample
 - d = depth of the trees fitted

In [152]:
maxnf = 8
maxnt = 50
maxd = 40

function tune_hyperparameters(maxnf, maxnt, maxd)
    accuracy = Array{Float64, 4}(maxnf, maxnt, 10, maxd)
    for nf in 1:maxnf, nt in 1:5:maxnt, p in 1:2:10, d in 1:2:maxd
        model_train = build_forest(labels_train, features_train, nf, nt, p/10, d)
        pred_cv = apply_forest(model_train, features_cv);
        a=sum(labels_cv.== pred_cv)/length(labels_cv)
        accuracy[nf, nt, p, d] =  a
    end
    return(accuracy)
end

accuracy = tune_hyperparameters(maxnf, maxnt, maxd);

Our 4-dimensional array 'accuracy' gets filled with accuracy measurements from fitting a random forest classifier with the respective set of hyperparameters to the cross-validation data set. The step size is greater than 1 in order to save some time. 

The next step extracts the first to fifth best fits and the respective parameters which we take the median of in order to reduce the risk of overfitting by just using the best fit as a simple form of regularization.

In order to get the 2nd, 3rd, ..., 5th best fit we delete the previous one. The collections idiom `1:end .!=idx` deselects the index at that place.

In [187]:
idx1 = collect(ind2sub(size(accuracy), indmax(accuracy)))

accuracy2nd = accuracy[1:end .!= idx1[1], 1:end .!=idx1[2], 1:end .!=idx1[3], 1:end .!=idx1[4]]
idx2 = collect(ind2sub(size(accuracy2nd), indmax(accuracy2nd)))

accuracy3rd = accuracy2nd[1:end .!= idx2[1], 1:end .!=idx2[2], 1:end .!=idx2[3], 1:end .!=idx2[4]]
idx3 = collect(ind2sub(size(accuracy3rd), indmax(accuracy3rd)))

accuracy4th = accuracy3rd[1:end .!= idx3[1], 1:end .!=idx3[2], 1:end .!=idx3[3], 1:end .!=idx3[4]]
idx4 = collect(ind2sub(size(accuracy4th), indmax(accuracy4th)))

accuracy5th = accuracy4th[1:end .!= idx4[1], 1:end .!=idx4[2], 1:end .!=idx4[3], 1:end .!=idx4[4]]
idx5 = collect(ind2sub(size(accuracy5th), indmax(accuracy5th)))

println("parameters: ", idx1)
println("performance: ", accuracy[idx1[1], idx1[2], idx1[3], idx1[4]])
println("parameters: ", idx2)
println("performance: ", accuracy2nd[idx2[1], idx2[2], idx2[3], idx2[4]])
println("parameters: ", idx3)
println("performance: ", accuracy3rd[idx3[1], idx3[2], idx3[3], idx3[4]])
println("parameters: ", idx4)
println("performance: ", accuracy4th[idx4[1], idx4[2], idx4[3], idx4[4]])
println("parameters: ", idx5)
println("performance: ", accuracy5th[idx5[1], idx5[2], idx5[3], idx5[4]])

parameters: [7, 31, 7, 15]
performance: 0.8661971830985915
parameters: [6, 45, 5, 26]
performance: 0.8591549295774648
parameters: [3, 26, 7, 20]
performance: 0.852112676056338
parameters: [2, 11, 1, 23]
performance: 0.8450704225352113
parameters: [2, 6, 2, 27]
performance: 0.8450704225352113


In [160]:
nf = round(Int, median([idx1[1], idx2[1], idx3[1], idx4[1], idx5[1]]));
nt = round(Int, median([idx1[2], idx2[2], idx3[2], idx4[2], idx5[2]]));
p = round(median([idx1[3], idx2[3], idx3[3], idx4[3], idx5[3]]))/10;
d = round(Int, median([idx1[4], idx2[4], idx3[4], idx4[4], idx5[4]]));

In [173]:
print("Number of features: ", nf, "\n",
      "Number of trees: ", nt, "\n",
      "Portion of the full sample: ", p, "\n",
      "Tree depth: ", d)

Number of features: 3
Number of trees: 26
Portion of the full sample: 0.5
Tree depth: 23

## Cross-validation test

K-fold cross-validation is performed with n = 10 number of folds (=splits of the original dataset) with training on a a subsample determined by p of the 9 folds and predicting the 10th.

In [182]:
acc = nfoldCV_forest(labels, features, nf, nt, 10, p);
IJulia.clear_output() ## to suppress all output

0

In [183]:
print("Average accuracy achieved: ", mean(acc), "\n",
      "Standard deviation of the performance: ", std(acc))

Average accuracy achieved: 0.7985915492957747
Standard deviation of the performance: 0.03639633322688132

In [164]:
histogram(acc,
          xaxis = ("Accuracy", (0,1)),
          legend = :none)

## Test on validation set

In [184]:
model_train = build_forest(labels_train, features_train, nf, nt, p, d)
pred_train = apply_forest(model_train, features_cv);
sum(labels_cv .== pred_train)/length(labels_cv)

0.8028169014084507

## Prediction on test set using all training examples

In [185]:
features_test = convert(Matrix, test[:,feature_list]);
model = build_forest(labels, features, nf, nt, p, d)
pred = apply_forest(model, features_test);

test[:Survived] = pred;
CSV.write("titanic_prediction.csv", test[[:PassengerId, :Survived]]);

This classifier scored accuracy = 0.794 on the corresponding Kaggle competition.